---
title: "Maggy Distributed Training with Tensorflow example"
date: 2021-05-03
type: technical_note
draft: false
---


Maggy enables you to train with Tensorflow distributed optimizers.
Using Maggy, you have to make minimal changes in train your model in a distributed fashion.

### 0. Spark Session

Make sure you have a running Spark Session/Context available.

On Hopsworks, just run your notebook to start the spark application.

### 1. Model definition

Let's define the model we want to train. The layers of the model have to be defined in the \_\_init__ function.

Do not instantiate the class, otherwise you won't be able to use Maggy.

In [1]:
import pyspark
sp = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("f-mnist-maggy") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.executor.cores", 4) \
    .config(
    "spark.dynamicAllocation.minExecutors","1") \
    .config("spark.dynamicAllocation.maxExecutors","5") \
    .getOrCreate()

sp

TypeError: an integer is required (got type bytes)

In [2]:
from tensorflow import keras 
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

# you can use keras.Sequential(), you just need to override it 
# on a custom class and define the layers in __init__()
class NeuralNetwork(Sequential):
        
    def __init__(self, nl=4):
        
        super().__init__()
        self.add(Dense(10,input_shape=(None,4),activation='tanh'))
        if nl >= 4:
          for i in range(0, nl-2):
            self.add(Dense(8,activation='tanh'))
        self.add(Dense(3,activation='softmax'))

model = NeuralNetwork

### 2. Dataset creation

You can create the dataset here and pass it to the TfDistributedConfig, or creating it in the training function.

You need to change the dataset path is correct.

In [3]:
train_set_path = "/Users/riccardo/Downloads/iris_train.csv"
test_set_path = "/Users/riccardo/Downloads/iris_test.csv"

In [4]:
train_set = sp.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(train_set_path).drop('_c0')

test_set = sp.read.format("csv").option("header","true")\
  .option("inferSchema", "true").load(test_set_path).drop('_c0')

raw_train_set = train_set.toPandas().values
raw_test_set = test_set.toPandas().values

In [5]:
def process_data(train_set, test_set):
    
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    
    X_train = train_set[:,0:4]
    y_train = train_set[:,4:]
    X_test = test_set[:,0:4]
    y_test = test_set[:,4:]

    return (X_train, y_train), (X_test, y_test)
  
train_set, test_set = process_data(raw_train_set, raw_test_set)

### 3. Defining the training function

The programming model is that you wrap the code containing the model training inside a wrapper function. Inside that wrapper function provide all imports and parts that make up your experiment.

The function should return the metric that you want to optimize for. This should coincide with the metric being reported in the Keras callback (see next point).
You can return the metric list, in this case only the loss element will be printed.

In [6]:
def hpo_function(number_layers, reporter):
  
  model = NeuralNetwork(nl=number_layers)
  model.build()
  
  #fitting the model and predicting
  model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
  train_input, test_input = process_data(raw_train_set, raw_test_set)

  train_batch_size = 75
  test_batch_size = 15
  epochs = 10
  
  model.fit(x=train_input[0], y=train_input[1],
            batch_size=train_batch_size,
            epochs=epochs,
            verbose=1)

  score = model.evaluate(x=test_input[0], y=test_input[1], batch_size=test_batch_size, verbose=1)
                         
  print(f'Test loss: {score[0]}')
  print(f'Test accuracy: {score[1]}')

  return score[1]

In [7]:
def training_function(model, train_set, test_set, hparams):
    
    model = model()
    model.build()
    #fitting the model and predicting

    model.compile(Adam(lr=hparams['learning_rate']),'categorical_crossentropy',metrics=['accuracy'])
    
    #raise ValueError(list(train_set.as_numpy_iterator()))

    model.fit(train_set,epochs=hparams['epochs'])

    accuracy = model.evaluate(test_set)

    return accuracy

### 4. Configuring the experiment

In order to use maggy distributed training, we have to configure the training model, we can pass it to TfDistributedConfig.
the model class has to be an implementation of __tf.keras.Model__.
We can also define __train_set__, __test_set__ and eventually the __model_parameters__. __model_parameters__ is a dictionary
containing the parameters to be used in the \_\_init__ function of your model.

In [8]:
from maggy.experiment_config.tf_distributed import TfDistributedConfig

#define the constructor parameters of your model
model_params = {
    #train dataset entries / num_workers
    'train_batch_size': 75,
    #test dataset entries / num_workers
    'test_batch_size': 15,
    'learning_rate': 0.04,
    'epochs': 20,
}

training_config = TfDistributedConfig(name="tf_test", model=model, train_set=train_set, test_set=test_set, process_data=process_data, hparams = model_params)

In [9]:
from maggy.experiment_config import OptimizationConfig
from maggy import Searchspace

# The searchspace can be instantiated with parameters
sp = Searchspace(number_layers=('INTEGER', [2, 8]))

hpo_config = OptimizationConfig(num_trials=4, optimizer="randomsearch", searchspace=sp, direction="max", es_interval=1, es_min=5, name="hp_tuning_test")

Hyperparameter added: number_layers


### 5. Run distributed training

Finally, we are ready to launch the maggy experiment. You just need to pass 2 parameters: the training function and the configuration variable we defined in the previous steps.

In [10]:
from maggy import experiment

result = experiment.lagom(train_fn=hpo_function, config=hpo_config)
result

Your are running Maggy on a base configuration.


AttributeError: 'NoneType' object has no attribute 'partitionId'

In [11]:
experiment.lagom(training_function, training_config)

Writing reports at /Users/riccardo/Git Repos/maggy/examples/experiment_log/local-1620719981928/2
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/20
INFO:tensorflow:Error reported to Coordinator: Layer tensorflow_model_wrapper expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor 'cond_1/Identity:0' shape=(None, 4) dtype=float32>, <tf.Tensor 'cond_1/Identity_1:0' shape=(None, 3) dtype=float32>]
Traceback (most recent call last):
  File "/Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/mirrored_run.py", line 323, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "/Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", l

ValueError: in user code:

    /Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/riccardo/miniforge3/envs/tf/lib/python3.8/site-packages/tensorflow/python/distribute/mirrored_strategy.py:628 _call_for_each_replica
        return mirrored_run.call_for_each